In [1]:
import cv2

# Entradas y salidas
video_input = 'noise.mp4'
video_output = 'video_salida_bilateral_cuda.mp4'

cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise IOError("No se pudo abrir el video de entrada.")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Codec para .mp4 (puede variar según sistema: 'mp4v', 'avc1', 'H264')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))

# Verificar soporte CUDA
if cv2.cuda.getCudaEnabledDeviceCount() == 0:
    raise Exception("CUDA no está disponible en OpenCV.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Subir frame a GPU
    gpu_frame = cv2.cuda_GpuMat()
    gpu_frame.upload(frame)

    # Separar canales B, G, R
    gpu_channels = cv2.cuda.split(gpu_frame)

    # Aplicar filtro bilateral a cada canal
    filtered_channels = []
    for ch in gpu_channels:
        filtered = cv2.cuda.bilateralFilter(ch, 9, 75, 75)
        filtered_channels.append(filtered)

    # Combinar canales nuevamente
    gpu_filtered_color = cv2.cuda.merge(filtered_channels)

    # Descargar a CPU
    filtered_frame = gpu_filtered_color


    out.write(filtered_frame)

    # Mostrar (opcional)
    cv2.imshow("CUDA Bilateral Filter - Color", filtered_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Limpiar
cap.release()
out.release()
cv2.destroyAllWindows()


In [3]:
import cv2

# Entradas y salidas
video_input = 'noise.mp4'
video_output = 'video_salida_clahe_cuda.mp4'

cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise IOError("No se pudo abrir el video de entrada.")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para .mp4
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))

# Verificar soporte CUDA
if cv2.cuda.getCudaEnabledDeviceCount() == 0:
    raise Exception("CUDA no está disponible en OpenCV.")

clahe = cv2.cuda.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gpu_frame = cv2.cuda_GpuMat()
    gpu_frame.upload(frame)

    gpu_ycrcb = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2YCrCb)
    channels = list(cv2.cuda.split(gpu_ycrcb))

    # Aplicar CLAHE al canal Y (luminosidad)
    clahe_y = clahe.apply(channels[0], stream=None)
    channels[0] = clahe_y

    merged = cv2.cuda.merge(channels)

    # Si merged no es GpuMat, subir a GPU
    if not isinstance(merged, cv2.cuda_GpuMat):
        gpu_ycrcb_clahe = cv2.cuda_GpuMat()
        gpu_ycrcb_clahe.upload(merged)
    else:
        gpu_ycrcb_clahe = merged

    gpu_result = cv2.cuda.cvtColor(gpu_ycrcb_clahe, cv2.COLOR_YCrCb2BGR)
    frame_result = gpu_result.download()

    out.write(frame_result)

    cv2.imshow("CUDA CLAHE", frame_result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
